In [2]:
import pandas as pd
import psycopg2 as pg2

In [3]:
conn = pg2.connect(
    dbname='adventureworks',
    user='postgres',
    password='galvanize',
    host='localhost',
    port='5558'
)

In [4]:
query = """
-- Define the threshold for vacation hours
WITH Threshold AS (
    SELECT 40 AS MinVacationHours
)

-- Main query to generate the HR report
SELECT
    CONCAT(Person.LastName, ', ', Person.FirstName) AS "Employee Name",
    Employee.JobTitle AS "Position",
    EmailAddress.EmailAddress AS "Email Address",
    Employee.VacationHours AS "Vacation Hours Accrued"
FROM
    HumanResources.Employee AS Employee
JOIN
    Person.Person AS Person ON Employee.BusinessEntityID = Person.BusinessEntityID
JOIN
    HumanResources.EmployeeDepartmentHistory AS EmployeeDeptHistory ON Employee.BusinessEntityID = EmployeeDeptHistory.BusinessEntityID
JOIN
    -- Subquery to get the latest start date for each employee
    (
        SELECT 
            BusinessEntityID, 
            MAX(StartDate) AS MaxStartDate
        FROM 
            HumanResources.EmployeeDepartmentHistory
        GROUP BY 
            BusinessEntityID
    ) AS LatestDeptHistory ON 
        EmployeeDeptHistory.BusinessEntityID = LatestDeptHistory.BusinessEntityID AND 
        EmployeeDeptHistory.StartDate = LatestDeptHistory.MaxStartDate
JOIN
    Person.EmailAddress AS EmailAddress ON Employee.BusinessEntityID = EmailAddress.BusinessEntityID
JOIN
    Threshold AS Threshold ON Employee.VacationHours > Threshold.MinVacationHours
ORDER BY
    "Employee Name";

"""

hr_df = pd.read_sql(query, conn)

conn.close()

C:\Users\darks\AppData\Local\Temp\ipykernel_5204\655294703.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hr_df = pd.read_sql(query, conn)


In [5]:
hr_df

,Employee Name,Position,Email Address,Vacation Hours Accrued
0,"Abolrous, Hazem",Quality Assurance Manager,hazem0@adventure-works.com,80
1,"Ackerman, Pilar",Shipping and Receiving Supervisor,pilar0@adventure-works.com,93
2,"Ajenstat, François",Database Administrator,françois0@adventure-works.com,67
3,"Alderson, Greg",Production Technician - WC45,greg0@adventure-works.com,85
4,"Alexander, Sean",Quality Assurance Technician,sean0@adventure-works.com,82
...,...,...,...,...
173,"Word, Sheela",Purchasing Manager,sheela0@adventure-works.com,49
174,"Wright, A. Scott",Master Scheduler,ascott0@adventure-works.com,44
175,"Wu, Peng",Quality Assurance Supervisor,peng0@adventure-works.com,81
176,"Yukish, Gary",Production Technician - WC40,gary0@adventure-works.com,69
